# Import Libraries

In [82]:
from lazypredict.Supervised import LazyClassifier
from scipy.io import arff
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import joblib
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Data 

## Loading Data

In [71]:
data,meta = arff.loadarff('Training Dataset.arff')
df = pd.DataFrame(data)

## Data Information

In [72]:
df = df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11055 entries, 0 to 11054
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   having_IP_Address            11055 non-null  object
 1   URL_Length                   11055 non-null  object
 2   Shortining_Service           11055 non-null  object
 3   having_At_Symbol             11055 non-null  object
 4   double_slash_redirecting     11055 non-null  object
 5   Prefix_Suffix                11055 non-null  object
 6   having_Sub_Domain            11055 non-null  object
 7   SSLfinal_State               11055 non-null  object
 8   Domain_registeration_length  11055 non-null  object
 9   Favicon                      11055 non-null  object
 10  port                         11055 non-null  object
 11  HTTPS_token                  11055 non-null  object
 12  Request_URL                  11055 non-null  object
 13  URL_of_Anchor                11

In [56]:
df.head()

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1


## Missing value

In [57]:
df.isnull().sum()

having_IP_Address              0
URL_Length                     0
Shortining_Service             0
having_At_Symbol               0
double_slash_redirecting       0
Prefix_Suffix                  0
having_Sub_Domain              0
SSLfinal_State                 0
Domain_registeration_length    0
Favicon                        0
port                           0
HTTPS_token                    0
Request_URL                    0
URL_of_Anchor                  0
Links_in_tags                  0
SFH                            0
Submitting_to_email            0
Abnormal_URL                   0
Redirect                       0
on_mouseover                   0
RightClick                     0
popUpWidnow                    0
Iframe                         0
age_of_domain                  0
DNSRecord                      0
web_traffic                    0
Page_Rank                      0
Google_Index                   0
Links_pointing_to_page         0
Statistical_report             0
Result    

# Preprocessing

## Chuẩn hóa dữ liệu

In [73]:
scaler = StandardScaler()

## Loại bỏ cột 'Result' và chuẩn hóa các đặc trưng

In [76]:
X = df.drop(columns=['Result'])  
X_scaled = scaler.fit_transform(X)

## Tách cột 'Result' làm biến mục tiêu

In [77]:
y = df['Result']

## Chia dữ liệu thành tập huấn luyện và kiểm tra

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [62]:
print(f"Tập huấn luyện: {X_train.shape}, Tập kiểm tra: {X_test.shape}")

Tập huấn luyện: (8844, 30), Tập kiểm tra: (2211, 30)


## Chuẩn hóa tập huấn luyện và tập kiểm tra

In [79]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Huấn luyện mô hình với LazyClassifier

## Khởi tạo LazyClassifier và huấn luyện các mô hình

In [80]:
lazy_classifier = LazyClassifier()
models, predictions = lazy_classifier.fit(X_train_scaled, X_test_scaled, y_train, y_test)
print(models)

100%|██████████| 31/31 [00:14<00:00,  2.13it/s]

[LightGBM] [Info] Number of positive: 4902, number of negative: 3942
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000792 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 98
[LightGBM] [Info] Number of data points in the train set: 8844, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.554274 -> initscore=0.217955
[LightGBM] [Info] Start training from score 0.217955
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
BaggingClassifier                  0.97               0.97     0.97      0.97   
ExtraTreesClassifier               0.97               0.97     0.97      0.97   
RandomForestClassifier             0.97               0.97     0.97      0.97   
LGBMClassifier                     0.97     

## Chọn mô hình tốt nhất theo Accuracy 

In [81]:
best_model = models.sort_values(by='Accuracy', ascending=False).iloc[0]
print(f"Tên mô hình tốt nhất: {best_model.name}")
print(f"Accuracy tốt nhất: {best_model['Accuracy']}")

Tên mô hình tốt nhất: BaggingClassifier
Accuracy tốt nhất: 0.9701492537313433


## Lưu mô hình tốt nhất vào tệp

In [66]:
best_model_trained = lazy_classifier.models[best_model.name]
joblib.dump(best_model_trained, 'best_model.pkl')
print("Mô hình đã được lưu.")

Mô hình đã được lưu.


## Khởi tạo mô hình 

In [83]:
bagging_model = BaggingClassifier(random_state=42)

### Xác định các giá trị hyperparameters cần tối ưu hóa

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_samples': [0.5, 0.8, 1.0],
    'max_features': [0.5, 0.8, 1.0],
    'bootstrap': [True, False]
}

### Khởi tạo và huấn luyện GridSearchCV

In [90]:
grid_search = GridSearchCV(estimator=bagging_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


GridSearchCV(cv=3, estimator=BaggingClassifier(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_features': [0.5, 0.8, 1.0],
                         'max_samples': [0.5, 0.8, 1.0],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy', verbose=1)

In [91]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

Best parameters found:  {'bootstrap': False, 'max_features': 0.8, 'max_samples': 0.8, 'n_estimators': 200}
Best cross-validation score:  0.9695838986883762


### Lấy mô hình tối ưu từ GridSearchCV

In [92]:
best_model = grid_search.best_estimator_

### Dự đoán với mô hình tối ưu

In [93]:
y_pred = best_model.predict(X_test_scaled)

In [94]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy của mô hình tối ưu: {accuracy}")

Accuracy của mô hình tối ưu: 0.9692446856625961


In [97]:
print(f"Accuracy ban đầu của mô hình ban đầu: 0.9698522761531504")
print(f"Accuracy của mô hình tối ưu: {accuracy}")

Accuracy ban đầu của mô hình ban đầu: 0.9698522761531504
Accuracy của mô hình tối ưu: 0.9692446856625961


### Lưu mô hình tối ưu

In [98]:
joblib.dump(best_model, 'optimized_model.pkl')
print("Mô hình tối ưu đã được lưu.")

Mô hình tối ưu đã được lưu.
